### 1. Configuração do Ambiente (Setup)

O primeiro passo é configurar nosso ambiente de desenvolvimento. Isso envolve a instalação de todas as bibliotecas (pacotes NuGet) necessárias para o projeto.

- **LangChain & LangChain.Providers.OpenAI**: O núcleo do framework LangChain e o provedor específico para interagir com os modelos da OpenAI.
- **Microsoft.Extensions.Caching.Memory**: Utilizado para implementações de cache em memória, que pode ser usado pelo LangChain para otimizar chamadas repetidas.
- **DotNetEnv**: Uma biblioteca auxiliar para carregar variáveis de ambiente de um arquivo `.env`, facilitando o gerenciamento de chaves de API e outras configurações sensíveis de forma segura.

In [55]:
#r "nuget: LangChain"
#r "nuget: LangChain.Providers.OpenAI"
#r "nuget: Microsoft.Extensions.Caching.Memory"
#r "nuget: Microsoft.Extensions.Caching.Abstractions"
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1 LangChain, 0.17.0 LangChain.Providers.OpenAI, 0.17.0 microsoft.extensions.caching.abstractions, 10.0.2 Microsoft.Extensions.Caching.Memory, 10.0.2

A seguir, carregamos a chave de API da OpenAI a partir de um arquivo `.env`. Armazenar a chave em um arquivo de variáveis de ambiente é uma boa prática de segurança para evitar que informações sensíveis sejam expostas diretamente no código.

In [56]:
using DotNetEnv;

var PathDocuments = System.Environment.GetFolderPath(System.Environment.SpecialFolder.MyDocuments);
var FilePath = System.IO.Path.Combine(PathDocuments, "estudos/langchan/.env");

Env.Load(FilePath);
string apiKey = Environment.GetEnvironmentVariable("API_KEY") ?? throw new InvalidOperationException("API_KEY not found in environment variables.");

### 2. Definição do Modelo e do Template de Prompt

Com o ambiente configurado, definimos os dois componentes principais para a nossa conversa:

1.  **Modelo (`model`)**: Instanciamos o `OpenAiLatestFastChatModel`, que representa o modelo de linguagem da OpenAI que irá gerar as respostas. Ele é inicializado com a chave de API que carregamos anteriormente.
2.  **Template de Prompt (`template`)**: O template é uma estrutura de texto que define como a entrada do usuário e o histórico da conversa serão formatados antes de serem enviados ao modelo.
    - `{history}`: É um placeholder onde o LangChain irá inserir o histórico da conversa.
    - `{input}`: É um placeholder para a nova entrada do usuário.

Essa estrutura ajuda o modelo a entender o contexto da conversa e a gerar respostas mais coerentes.

In [57]:
using LangChain.Providers.OpenAI.Predefined;

var model = new OpenAiLatestFastChatModel(apiKey);
var template = @"Você é um programador experiente em C# que está apoiando um desenvolvedor Senior nos testes.
{history}
Usuário: {input}
Programador experiente:";

### 3. Implementação da Memória Conversacional

Para que o chatbot "lembre" das interações passadas, implementamos uma estratégia de memória. O LangChain oferece várias estratégias, e aqui estamos construindo uma `ConversationBufferMemory`.

- **`GetBaseChatMessageHistory()`**: Cria uma instância de `ChatMessageHistory`, que é um armazenamento em memória simples para as mensagens da conversa.
- **`GetConversationBufferMemory(...)`**: Cria a `ConversationBufferMemory`, que utiliza o `ChatMessageHistory` para guardar as mensagens. Também configuramos um `MessageFormatter` para definir prefixos customizados para as mensagens do usuário ("Usuário:") e da IA ("Programador experiente:").
- **`PickMemoryStrategy(...)`**: Junta todas as peças, criando e retornando a estratégia de memória configurada que será usada pela nossa cadeia.

In [58]:
private static BaseChatMemory GetConversationBufferMemory(BaseChatMessageHistory chatHistory, MessageFormatter messageFormatter)
    {
        return new ConversationBufferMemory(chatHistory)
        {
            Formatter = messageFormatter
        };
    }

In [59]:
using LangChain.Memory;

private static BaseChatMessageHistory GetBaseChatMessageHistory()
{
    return new ChatMessageHistory();
}

In [60]:
using LangChain.Memory;
using LangChain.Providers;

private static BaseChatMemory PickMemoryStrategy(IChatModel model)
{
    MessageFormatter formatter = new MessageFormatter
    {
        AiPrefix = "Programador experiente",
        HumanPrefix = "Usuário"
    };

    BaseChatMessageHistory chatHistory = GetBaseChatMessageHistory();

    return GetConversationBufferMemory(chatHistory, formatter);
}

### 4. Composição da Cadeia (Chain)

Agora, montamos a "cadeia" (chain), que é a sequência de passos que serão executados em ordem. Utilizamos o operador `|` para conectar cada componente:

1.  **`LoadMemory(...)`**: O primeiro passo carrega o histórico da conversa da memória (usando a estratégia que definimos) e o insere na variável `history`, que será usada pelo template.
2.  **`Template(...)`**: Formata o prompt com o histórico e a nova entrada do usuário.
3.  **`LLM(...)`**: Envia o prompt formatado para o modelo da OpenAI. A resposta do modelo será, por padrão, colocada na variável `text`.
4.  **`UpdateMemory(...)`**: O último passo atualiza a memória da conversa, salvando a pergunta do usuário (da variável `input`) e a resposta da IA (da variável `text`) para serem usadas nas próximas interações.

In [61]:
using static LangChain.Chains.Chain;

var chain = 
    LoadMemory(PickMemoryStrategy(model), "history")
    | Template(template)
    | LLM(model)
    | UpdateMemory(PickMemoryStrategy(model), requestKey: "input", responseKey: "text");

### 5. Execução da Cadeia

Finalmente, executamos a cadeia.

1.  **`Set("Ola como vc está:", "input")`**: Primeiro, usamos a cadeia `Set` para definir a pergunta inicial do usuário. O texto "Ola como vc está:" é colocado na variável `input`.
2.  **`| chain`**: Conectamos esta entrada à cadeia principal que construímos no passo anterior.
3.  **`finalChain.RunAsync("text")`**: Executamos a cadeia completa de forma assíncrona e pedimos o valor da variável de saída `text`, que conterá a resposta gerada pelo modelo.
4.  **`Console.WriteLine(result)`**: Imprimimos o resultado no console.

In [62]:
var finalChain = Set("Ola como vc está:", "input") | chain;
var result = await finalChain.RunAsync("text");

Console.WriteLine(result);